## Everyone vs Everyone
What if instead of pairing up each week, everyone played everyone. Who would win?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
from base_python_files.api_info import *
from base_python_files.credentials import *
from base_python_files.api_functions import *
from base_python_files.stat_functions import *

In [ ]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

Generate a matrix of winrates across all the weeks, where each row/column is a team.


In [ ]:
team_week_scores = {}
team_matrix = np.zeros((8,8))
min_weeks = 0
weeks = 22
for week in range(min_weeks,weeks):
    team_scores = retrieve_espn_matchup(swid,espn_s2,league_id,week=week)
    for team_id in team_scores:
        wins = 0
        draws = 0
        losses = 0
        total_cat_wins = 0
        total_cat_losses = 0
        total_cat_draws = 0
        for second_team_id in team_scores:
            if team_id==second_team_id:
                continue
            cats_won = 0
            cats_lost = 0
            team = team_scores[team_id]
            second_team = team_scores[second_team_id]
            for score_cat in team:
                if team[score_cat]>second_team[score_cat]:
                    cats_won+=1
                elif team[score_cat]<second_team[score_cat]:
                    cats_lost+=1
            total_cat_wins+=cats_won
            total_cat_losses+=cats_lost
            total_cat_draws+=13-cats_won-cats_lost
            if cats_won>cats_lost:
                wins+=1
                team_matrix[team_id-1,second_team_id-1]+=1
                team_matrix[second_team_id-1,team_id-1]-=1
            elif cats_won<cats_lost:
                losses+=1
                team_matrix[team_id-1,second_team_id-1]-=1
                team_matrix[second_team_id-1,team_id-1]+=1
            else:
                draws+=1
            # if week == 15 and team_id==8:
                # print('ID', second_team_id,'Winning',cats_won,'Losing',cats_lost)
        if team_id in team_week_scores:
            team_week_scores[team_id]+=np.array([wins,losses,draws,total_cat_wins,total_cat_losses,total_cat_draws])
        else:
            team_week_scores[team_id]=np.array([wins,losses,draws,total_cat_wins,total_cat_losses,total_cat_draws])

team_matrix/=(weeks-1-min_weeks+1)*2      
    

In [ ]:
plt.imshow(team_matrix)
plt.colorbar()
plt.suptitle(f"Vs table for the first {weeks} weeks showing win rates for the attacking team.")
plt.title(f"0=draw on average, 1=always win, -1=always lose")
plt.xlabel("Defending Team")
plt.ylabel("Attacking Team")

In [ ]:
team_week_infos = {}
for team in team_week_scores:
    team_week_infos[team]={
        "wins":team_week_scores[team][0],
        "losses":team_week_scores[team][1],
        "draws":team_week_scores[team][2],
        "win percentage":team_week_scores[team][0]/(team_week_scores[team][0]+team_week_scores[team][1]+team_week_scores[team][2]),
        "win or draw percentage":(team_week_scores[team][0]+team_week_scores[team][2])/(team_week_scores[team][0]+team_week_scores[team][1]+team_week_scores[team][2]),
        "category wins":team_week_scores[team][3],
        "category losses":team_week_scores[team][4],
        "category draws":team_week_scores[team][5],
        "category win percentage":team_week_scores[team][3]/(team_week_scores[team][3]+team_week_scores[team][4]+team_week_scores[team][5]),
        "category win or draw percentage":(team_week_scores[team][3]+team_week_scores[team][5])/(team_week_scores[team][3]+team_week_scores[team][4]+team_week_scores[team][5]),

    }

In [ ]:
sorting_key = "win percentage" 
ordered_scores = [f'Team: {x[0]}, {sorting_key}: {x[1][sorting_key]}' for x in sorted(team_week_infos.items(),key=lambda x:x[1][sorting_key],reverse=True)]
ordered_scores

In [ ]:
sorting_key = "win or draw percentage" 
ordered_scores = [f'Team: {x[0]}, {sorting_key}: {x[1][sorting_key]}' for x in sorted(team_week_infos.items(),key=lambda x:x[1][sorting_key],reverse=True)]
ordered_scores

In [ ]:
sorting_key = "category win percentage"
ordered_scores = [f'Team: {x[0]}, {sorting_key}: {x[1][sorting_key]}' for x in sorted(team_week_infos.items(),key=lambda x:x[1][sorting_key],reverse=True)]
ordered_scores

In [ ]:
sorting_key = "category win or draw percentage"
ordered_scores = [f'Team: {x[0]}, {sorting_key}: {x[1][sorting_key]}' for x in sorted(team_week_infos.items(),key=lambda x:x[1][sorting_key],reverse=True)]
ordered_scores

### What are each player's best scores for each category?

In [ ]:
best_scores = {p:{} for p in range(1,9)}
min_weeks = 1
weeks = 18
for g_team_id in range(1,9):
    cat_wins = {}
    for week in range(min_weeks,weeks):
        team_scores = retrieve_espn_matchup(swid,espn_s2,league_id,week=week)
        for opp_team_id in range(1,9):
            opp_team = team_scores[opp_team_id]
            for cat in opp_team:
                if cat not in best_scores[opp_team_id] or opp_team[cat] > best_scores[opp_team_id][cat][0]:
                    best_scores[opp_team_id][cat]=(opp_team[cat],week)

In [ ]:
best_scores